In [ ]:
import os
import numpy as np

def read_label_map(label_map_path):

    item_id = None
    item_name = None
    items = {}
    
    class_labels = np.sort(os.listdir(label_map_path))[3:]
    for i, label in enumerate(class_labels):
        items[i] = label
    items[i+1] = "happens"

    return items


items = read_label_map("/mnt/tmp/JHMDB/Frames")

# main_path = '/tmp2'
images_path = '/mnt/tmp/JHMDB/Frames'
vis_save_path = './output_images'
 
# TubeR's csn152, ava21 results
detection = '../tmp_jhmdb/0.txt' #numbers are changeable
gt = '../tmp_jhmdb/GT_0.txt'

query_num = 15

In [ ]:
anno_dict = {}
with open(gt) as f:
    for line in f.readlines():
        img_id = line.split(' ')[0]
        annotation = [int(float(n)) for n in line.split('[')[1].split(']')[0].split(',')]
        one_hot_obj_label = annotation[6:]
        obj_labels = [items[i] for i, e in enumerate(one_hot_obj_label) if e!=0]
        coord = annotation[2:6]
        if img_id not in anno_dict.keys():
            anno_dict[img_id] = {
                "obj": [obj_labels],
                "coord": [coord]
            }
        else:
            anno_dict[img_id]["obj"].append(obj_labels)
            anno_dict[img_id]["coord"].append(coord)

# print("test if anno_dict is well constructed: ", anno_dict['_dBTTYDRdRQ_1528'])

pred_dict = {}
with open(detection) as f:
    for line in f.readlines(): #[0::15]:
        img_id = line.split(' ')[0]
        # print(line)
        annotation = [float(n) for n in line.split('[')[1].split(']')[0].split(',')]
        one_hot_obj_label = annotation[4:-1]
        # print(len(one_hot_obj_label))
        obj_labels = [items[i] for i, e in enumerate([k>0.7 for k in one_hot_obj_label]) if e]
        # print([k>1e-30 for k in multi_hot_obj_label])
        if len(obj_labels)==0:
            continue
        coord = annotation[:4]
        if img_id not in pred_dict.keys():
            pred_dict[img_id] = {
                "obj": [obj_labels],
                "coord": [coord]
            }
        else:
            pred_dict[img_id]["obj"].append(obj_labels)
            pred_dict[img_id]["coord"].append(coord)


In [ ]:
# from IPython.display import Image
import PIL
import os
import torch
# import sys
# module_path = "./visualization_utils.py"
# sys.path.append(module_path)
import visualization_utils_custom as vis_utils
import datasets.video_transforms as T
import torchvision.transforms.functional as F
import models.transformer.util.box_ops as box_ops

%matplotlib inline
number = 0
well_localized = 0
exclude_counts = 0
misclassified = 0
for image in list(anno_dict.keys())[::2]:
    temp_cls = "_".join(image.split('_')[:2])
    classes_ = ["brush_hair", "kick_ball", "shoot_ball", "shoot_bow", "shoot_gun", "climb_stairs", "swing_baseball"]
    cls = temp_cls if temp_cls in classes_ else temp_cls.split('_')[0]
    vid = "_".join(image.split('_')[2:]) if temp_cls in classes_ else "_".join(image.split('_')[1:])
    vid = "-".join(vid.split('-')[:-1])
    image_num = image.split('-')[-1]
    img_path = os.path.join(images_path, cls, vid, str(int(image_num)).zfill(5)+".png")
    img = PIL.Image.open(img_path)
    w, h = img.size
    if w >= h:
        nh = 224
        nw = 224 * w / h
    else:
        nw = 224
        nh = 224 * h / w

    # crop_top = int(round((h - nh) / 2.))
    # crop_left = int(round((w - nw) / 2.))

    img = img.resize((int(nw), int(nh)))
    # img = F.crop(img, crop_top, crop_left, nh, nw)
    
    try:
        num_gt_instances = len(anno_dict[image]["obj"])
        for i, (gt_obj, pred_obj, gt_coord, pred_coord) in enumerate(zip(anno_dict[image]["obj"], pred_dict[image]["obj"], anno_dict[image]["coord"], pred_dict[image]["coord"])):
            gt_cat = str(gt_obj)
            pred_cat = str(pred_obj)
            gt_xmin, gt_ymin, gt_xmax, gt_ymax = gt_coord[0], gt_coord[1], gt_coord[2], gt_coord[3]
            pred_xmin, pred_ymin, pred_xmax, pred_ymax = pred_coord[0], pred_coord[1], pred_coord[2], pred_coord[3]
            vis_utils.draw_bounding_box_on_image(
                img, gt_ymin, gt_xmin, gt_ymax, gt_xmax,
                color = 'Green',
                display_str_list=[gt_cat],
                use_normalized_coordinates=False
            )
            vis_utils.draw_bounding_box_on_image(
                img, pred_ymin, pred_xmin, pred_ymax, pred_xmax,
                color = 'Yellow',
                display_str_list=[pred_cat],
                use_normalized_coordinates=False,
                margin2=20
            )            
        # iou = box_ops.box_iou(torch.tensor([gt_xmin, gt_ymin, gt_xmax, gt_ymax]).unsqueeze(0), torch.tensor([pred_xmin, pred_ymin, pred_xmax, pred_ymax]).unsqueeze(0))[0]
        # if iou >= 0.5:
        #     # print("localized well")
        #     well_localized += 1
        if pred_cat != gt_cat:
            print("image id: ", image)
            display(img)
            misclassified += 1
        # else:
        #     print("image id: ", image)
        #     display(img)
        #     continue
        # print(number)
        # print("number of GT instances: ", num_gt_instances)
        # print("number of predictions: ", num_predicted_instances)
        number += 1
        # del img
    except:
        # print("excluded key frame encountered, skip to the next frame")
        exclude_counts += 1
        # user_input = str(input())
        # if user_input == "q": break
        # else: pass
print(misclassified, len(anno_dict.keys())-exclude_counts)

In [ ]:
box_ops.box_iou(torch.tensor([gt_xmin, gt_ymin, gt_xmax, gt_ymax]).unsqueeze(0), torch.tensor([pred_xmin, pred_ymin, pred_xmax, pred_ymax]).unsqueeze(0))[0] > 0.5

In [ ]:
# from IPython.display import Image
import PIL
import os
# import sys
# module_path = "./visualization_utils.py"
# sys.path.append(module_path)
import visualization_utils_custom as vis_utils
%matplotlib inline
number = 0
for image in list(anno_dict.keys())[:1500]:
    temp_cls = "_".join(image.split('_')[:2])
    classes_ = ["brush_hair", "kick_ball", "shoot_ball", "shoot_bow", "shoot_gun", "climb_stairs", "swing_baseball"]
    cls = temp_cls if temp_cls in classes_ else temp_cls.split('_')[0]
    vid = "_".join(image.split('_')[2:]) if temp_cls in classes_ else "_".join(image.split('_')[1:])
    vid = "-".join(vid.split('-')[:-1])
    image_num = image.split('-')[-1]
    img_path = os.path.join(images_path, cls, vid, str(int(image_num)).zfill(5)+".png")
    img = PIL.Image.open(img_path)
    w, h = img.size
    if w >= h:
        nh = 224
        nw = 224 * w / h
    else:
        nw = 224
        nh = 224 * h / w
    
    img = img.resize((int(nw), int(nh)))

    try:
        num_gt_instances = len(anno_dict[image]["obj"])
        for i, (obj, coord) in enumerate(zip(anno_dict[image]["obj"], anno_dict[image]["coord"])):
            cat = str(obj)
            if ('shoot' in cat): #or ('crouch' in cat) or ('grab (a person)' in cat) or ('smoke' in cat):
                xmin, ymin, xmax, ymax = coord[0], coord[1], coord[2], coord[3]
                vis_utils.draw_bounding_box_on_image(
                    img, ymin, xmin, ymax, xmax,
                    color = 'Green',
                    display_str_list=[cat],
                    use_normalized_coordinates=False
                )
                print_pred = True
                print("image id: ", image)
            else:
                print_pred = False
        if print_pred:
            for j, (obj, coord) in enumerate(zip(pred_dict[image]["obj"], pred_dict[image]["coord"])):
                cat = str(obj)
                xmin, ymin, xmax, ymax = coord[0], coord[1], coord[2], coord[3]
                vis_utils.draw_bounding_box_on_image(
                    img, ymin, xmin, ymax, xmax,
                    color = 'Yellow',
                    display_str_list=[cat],
                    use_normalized_coordinates=False,
                    margin2=20
                )
            display(img)
            print(number)
        # print("number of GT instances: ", num_gt_instances)
        # print("number of predictions: ", num_predicted_instances)
            number += 1
        # del img
    except:
        print("excluded key frame encountered, skip to the next frame")
        # user_input = str(input())
        # if user_input == "q": break
        # else: pass

In [ ]:
str(anno_dict[image]['obj'][0])

In [ ]:
anno_dict[image]['coord'][0]

In [ ]:
len(anno_dict.keys())